---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    
    # Use linearity of log-log plot of degree distribution to determine whether PA or SW.
    # If SW, use average clustering to determine wheter SW_L or SW_H.
    
    r_sq = [0 for item in range(len(P1_Graphs))]
    for i in range(len(r_sq)):
        degree_sequence=sorted(nx.degree(P1_Graphs[i]).values(),reverse=True)
        coeffs = np.polyfit(
            np.log(np.arange(len(degree_sequence)) + 1),
            np.log(degree_sequence), 
            1
        )
        coeffs = list(coeffs)
        r = np.corrcoef(
            np.log(np.arange(len(degree_sequence)) + 1),
            np.log(degree_sequence)
        )[0, 1]
        r_sq[i] = r**2
    
    avg_clustering           = [nx.average_clustering(P1_Graphs[item]) for item in range(len(P1_Graphs))]
    
    graph_types = ['' for item in range(len(P1_Graphs))]
    for i in range(len(graph_types)):
        if r_sq[i] > 0.9:
            graph_types[i] = 'PA'
        else:
            if avg_clustering[i] > 0.1:
                graph_types[i] = 'SW_L'
            else:
                graph_types[i] = 'SW_H'
    
    return graph_types # Your Answer Here

In [4]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [91]:
def salary_predictions():
    
    # Your Code Here
    
    # load node properties into data frame
    df = pd.DataFrame()
    df['node']                   = pd.Series(G.nodes())
    df['degree']                 = pd.Series(nx.degree(G))
    df['degree_centrality']      = pd.Series(nx.degree_centrality(G))
    df['closeness_centrality']   = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweenness_centrality'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['clustering']             = pd.Series(nx.clustering(G))
    df['page_rank']              = pd.Series(nx.pagerank(G))
    df['department']             = pd.Series([G.nodes(data=True)[item][1]['Department'] for item in range(len(G.nodes()))])
    df['management_salary']      = pd.Series([G.nodes(data=True)[item][1]['ManagementSalary'] for item in range(len(G.nodes()))])
    
    df.set_index('node')
    
    # use rows for which management salary is classified for labelled data
    df_labelled   = df[~(df['management_salary'].isnull())] # remove rows with unknown management salaries
#     df_labelled   = df_labelled.reset_index()
    
    # use rows for which management salary is not classified for unlabelled data
    df_unlabelled = df[df['management_salary'].isnull()]
#     df_unlabelled = df_unlabelled.reset_index()
    
    from sklearn.ensemble import GradientBoostingClassifier
#     from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler
    
    X_labelled   = df_labelled[['degree_centrality', 'closeness_centrality', 'betweenness_centrality', 'clustering']]
    y_labelled   = df_labelled[['management_salary']]#.values.ravel()
    X_unlabelled = df_unlabelled[['degree_centrality', 'closeness_centrality', 'betweenness_centrality', 'clustering']]
    y_unlabelled = df_unlabelled[['management_salary']]#.values.ravel()
    
#     X_train, X_test, y_train, y_test = train_test_split(np.array(X_labelled), np.array(y_labelled)) # For looking at algorithm accuracy while testing
#     X_train, X_test = np.array(X_labelled), np.array(X_unlabelled)
#     y_train, y_test = y_labelled.astype(int), y_unlabelled.astype(int)
    
    X_train, y_train, X_test, y_test = X_labelled, y_labelled, X_unlabelled, y_unlabelled
    
    X_train, X_test = MinMaxScaler().fit_transform(X_train), MinMaxScaler().fit_transform(X_test)
    
    clf = GradientBoostingClassifier(max_depth=4)
    clf.fit(X_train, y_train)
    y_test_probabilities = clf.predict_proba(X_test)
    y_vals = clf.predict(X_test)
    y_test_probabilities = y_test_probabilities[:, 1]
    df_unlabelled['management_salary_probabilities'] = y_test_probabilities
    
    probs = pd.Series(df_unlabelled['management_salary_probabilities']) # generate from above probabilities, with index equal to node
    
    return probs#, df, df_labelled, df_unlabelled, y_vals # Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [94]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [116]:
def new_connections_predictions():
    
    # Your Code Here
    
        # load node properties into data frame
    df = future_connections
    
    preferential_attachment = lambda edge: list(nx.preferential_attachment(G, [edge]))[0][2]
    n_common_neighbors      = lambda edge: len(list(nx.common_neighbors(G, edge[0], edge[1])))
    
    df['preferential_attachment'] = [preferential_attachment(item) for item in future_connections.index]
    df['n_common_neighbors'] = [n_common_neighbors(item) for item in future_connections.index]
    
    df_labelled = df[~(df['Future Connection'].isnull())]
    df_unlabelled = df[(df['Future Connection'].isnull())]
    
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.preprocessing import MinMaxScaler
    
    X_labelled   = df_labelled[['preferential_attachment', 'n_common_neighbors']]
    y_labelled   = df_labelled[['Future Connection']]
    X_unlabelled = df_unlabelled[['preferential_attachment', 'n_common_neighbors']]
    y_unlabelled = df_unlabelled[['Future Connection']]
    
    X_train, y_train, X_test, y_test = X_labelled, y_labelled, X_unlabelled, y_unlabelled
    
    X_train, X_test = MinMaxScaler().fit_transform(X_train), MinMaxScaler().fit_transform(X_test)
    
    clf = GradientBoostingClassifier()
    clf.fit(X_train, y_train)
    y_test_probabilities = clf.predict_proba(X_test)
    # y_vals = clf.predict(X_test)
    y_test_probabilities = y_test_probabilities[:, 1]
    df_unlabelled['future_connection_probability'] = y_test_probabilities
    
    probs = pd.Series(df_unlabelled['future_connection_probability']) # generate from above probabilities, with index equal to node
    
    return probs #df, df_labelled, df_unlabelled # Your Answer Here

In [117]:
# df_1, df_labelled_1, df_unlabelled_1 = new_connections_predictions()
blah = new_connections_predictions()
blah.head(15)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.055304
(542, 751)    0.012931
(20, 426)     0.935342
(50, 989)     0.013104
(942, 986)    0.013750
(324, 857)    0.012887
(13, 710)     0.288593
(19, 271)     0.275696
(319, 878)    0.013104
(659, 707)    0.012931
(49, 843)     0.013104
(208, 893)    0.012931
(377, 469)    0.012482
(405, 999)    0.022630
(129, 740)    0.017730
Name: future_connection_probability, dtype: float64